In [ ]:
! pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 8.5MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=93c531d7068bdbb6db314d55c1eddb62228b11b9c314ac7e582ae05c56e79efa
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d sanikamal/rock-paper-scissors-dataset

In [ ]:
! unzip rock-paper-scissors-dataset.zip -d dataset

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
training_path = "/content/dataset/Rock-Paper-Scissors/train"
test_path = "/content/dataset/Rock-Paper-Scissors/test"

In [ ]:
dataGen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range=5,
    horizontal_flip=True,
    vertical_flip=True,
)

In [ ]:
train_dataset_aug = dataGen.flow_from_directory(
    training_path,
    target_size = (200,200),
    batch_size = 32,
    shuffle=True,
    seed = 865
)

Found 2520 images belonging to 3 classes.


In [ ]:
test_dataset_aug = dataGen.flow_from_directory(
    test_path,
    target_size = (200,200),
    batch_size = 32,
    shuffle=True,
    seed = 865
)

Found 372 images belonging to 3 classes.


# Using Pretrained model (MobileNet)

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
pretrained_model.trainable = False

In [ ]:
pretrained_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [ ]:
model = tf.keras.models.Sequential()

In [ ]:
model.add(pretrained_model)

In [ ]:
model.add(tf.keras.layers.GlobalAveragePooling2D())

In [ ]:
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Compile
model.compile(optimizer = 'adam',
                loss='categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(train_dataset_aug, validation_data=test_dataset_aug, epochs = 5)

Epoch 1/5
79/79 [==============================] - 64s 381ms/step - loss: 0.5916 - accuracy: 0.7661 - val_loss: 0.3624 - val_accuracy: 0.8763
Epoch 2/5
79/79 [==============================] - 29s 367ms/step - loss: 0.0232 - accuracy: 0.9932 - val_loss: 0.1420 - val_accuracy: 0.9516
Epoch 3/5
79/79 [==============================] - 29s 369ms/step - loss: 0.0139 - accuracy: 0.9946 - val_loss: 0.3449 - val_accuracy: 0.8952
Epoch 4/5
79/79 [==============================] - 29s 367ms/step - loss: 0.0047 - accuracy: 0.9990 - val_loss: 0.2467 - val_accuracy: 0.9274
Epoch 5/5
79/79 [==============================] - 29s 369ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.2696 - val_accuracy: 0.9247


In [ ]:
model.save('model_trained.h5')